In [52]:
import numpy as np
from glob import glob
import pandas as pd
from natsort import natsorted

## Figure 1

In [57]:
pump_powers = np.array([2000000.0,3000000.0,4000000.0,5000000.0,6000000.0,7000000.0,8000000.0,9000000.0,10000000])
auc_to_mJ = 1.60324933931e-07
pump_powers_mJ = ['%.1f mJ' % (float(p) * auc_to_mJ,) for p in pump_powers]

iq_vs_power_data = {}
iq_vs_power_CIs  = {}

for power_csv in glob("../csvs/water_Iq_v_power_??fs.csv") + glob("../csvs/water_Iq_v_power_???fs.csv"):
    print(f"loading {power_csv}")
    data = np.loadtxt(power_csv, delimiter=',')
    delay = int(power_csv.split("_")[-1].split(".")[0][:-2])
    iq_vs_power_data[f"{delay} fs"] = data[:,1:]

for power_csv in glob("../csvs/water_Iq_v_power_95percentCI_*fs.csv"):
    print(f"loading {power_csv}")
    data = np.loadtxt(power_csv, delimiter=',')
    delay = int(power_csv.split("_")[-1].split(".")[0][:-2])
    iq_vs_power_CIs[f"{delay} fs"] = data[:,1:]

loading ../csvs/water_Iq_v_power_50fs.csv
loading ../csvs/water_Iq_v_power_90fs.csv
loading ../csvs/water_Iq_v_power_70fs.csv
loading ../csvs/water_Iq_v_power_30fs.csv
loading ../csvs/water_Iq_v_power_10fs.csv
loading ../csvs/water_Iq_v_power_40fs.csv
loading ../csvs/water_Iq_v_power_80fs.csv
loading ../csvs/water_Iq_v_power_60fs.csv
loading ../csvs/water_Iq_v_power_20fs.csv
loading ../csvs/water_Iq_v_power_110fs.csv
loading ../csvs/water_Iq_v_power_100fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_20fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_60fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_40fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_80fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_100fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_10fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_30fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_70fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_110fs.csv
loading ../csvs/water_Iq_v_power_95percentCI_50f

In [59]:
with pd.ExcelWriter("./excel/fig1_experimental_Iq.xlsx") as writer:
    for delay in natsorted(iq_vs_power_data.keys()):
        pd.DataFrame(iq_vs_power_data[delay], columns=["q 1/A"] + pump_powers_mJ).to_excel(writer, sheet_name=f'{delay} fs delay', index=False)

with pd.ExcelWriter("./excel/fig1_experimental_Iq_95CI.xlsx") as writer:
    for delay in natsorted(iq_vs_power_data.keys()):
        pd.DataFrame(iq_vs_power_CIs[delay], columns=["q 1/A"] + pump_powers_mJ).to_excel(writer, sheet_name=f'{delay} fs delay', index=False)

# Figure 2

In [19]:
# panels: d e f h 

csp_010 = np.genfromtxt("./charge_populations/csp47_010_single.txt")
csp_100 = np.genfromtxt("./charge_populations/csp47_100_single.txt")

ion_010 = np.genfromtxt("./charge_populations/ion47_010_single.txt")
ion_100 = np.genfromtxt("./charge_populations/ion47_100_single.txt")

def format_csp_column_names(prefix, length):
    column_names = ["time (fs)",] + [prefix + " +%d" % i for i in range(length)]
    return column_names

with pd.ExcelWriter("./excel/fig2_charge_states_and_species.xlsx") as writer:
    pd.DataFrame(csp_010, columns=format_csp_column_names("10% fluence", csp_010.shape[1]-1)).to_excel(writer, sheet_name='10% fluence charge species', index=False)
    pd.DataFrame(csp_100, columns=format_csp_column_names("100% fluence", csp_100.shape[1]-1)).to_excel(writer, sheet_name='100% fluence charge species', index=False)
    pd.DataFrame(ion_010, columns=["time (fs)", "10% fluence charge"]).to_excel(writer, sheet_name='10% fluence total charge', index=False)
    pd.DataFrame(ion_100, columns=["time (fs)", "100% fluence charge"]).to_excel(writer, sheet_name='100% fluence total charge', index=False)

In [20]:
# panels: g i

THEORY_POWERS = [10, 50, 100]
THEORY_DELAYS = [10, 30, 50, 70, 90, 110]
THEORY_INTEGRATING_DELAYS = [25, 40, 75]

def load_theory_result(power, delays=THEORY_DELAYS):

    dirmap = {
        10: "Iq_010",
        50: "Iq_050",
        100: "Iq_100",
    }

    q_values = None
    intensities = []

    for delay in delays:
        file_path = f"{dirmap[power]}/rdp47_{power:03d}_{delay:03d}fs.txt"
        data = np.loadtxt(file_path)

        if q_values is not None:
            assert np.all(q_values == data[:,0] / 1e10)
        else:
            q_values = data[:,0] / 1e10

        intensities.append(data[:,1])

    data_stack = np.stack([q_values] + intensities)
    assert data_stack.shape[0] == len(delays) + 1

    return data_stack.T


delay_labels = ["q 1/A"] + [f"{delay} fs" for delay in THEORY_DELAYS]


with pd.ExcelWriter("./excel/fig2_theory_Iq.xlsx") as writer:
    for power in THEORY_POWERS:
        theory_Iq = load_theory_result(power)
        pd.DataFrame(theory_Iq, columns=delay_labels).to_excel(writer, sheet_name=f"{power}% fluence", index=False)

# Figure 3

In [33]:
# 1st column: time [fs] time = 0 fs at the pump pulse center
# 2nd column: electron concentration n_e [m^-3]
# 3rd column: electron temperature T_e [eV]
# 4th column: oxygen ion temperature T_i [eV]

fig3_column_names = ["time (fs)", "e- conc [m^-3]", "e- temp [eV]", "oxygen ion temp [eV]"]

low_fluence_temperatures = np.loadtxt("./temperature-data/noprobe_010.txt")
high_fluence_temperatures = np.loadtxt("./temperature-data/noprobe_100.txt")

# convert from 1/m3 to 1/cm3
low_fluence_temperatures[:,1] *= 1e-6
high_fluence_temperatures[:,1] *= 1e-6

In [34]:
with pd.ExcelWriter("./excel/fig3_plasma_parameters.xlsx") as writer:
    pd.DataFrame(low_fluence_temperatures, columns=fig3_column_names).to_excel(writer, sheet_name=f"10% fluence", index=False)
    pd.DataFrame(high_fluence_temperatures, columns=fig3_column_names).to_excel(writer, sheet_name=f"100% fluence", index=False)

## Figure 4

In [30]:
def load_factor_block(fluence):

    data = [
        np.genfromtxt("./scattering_effect_decomposition/rdp47_undam_water.txt")[:,0] / 1e10,
        np.genfromtxt("./scattering_effect_decomposition/rdp47_undam_water.txt")[:,1],
        np.genfromtxt(f"./scattering_effect_decomposition/rdp47_{fluence}_disponly.txt")[:,1],
        np.genfromtxt(f"./scattering_effect_decomposition/rdp47_{fluence}_formfonly.txt")[:,1],
        np.genfromtxt(f"./scattering_effect_decomposition/rdp47_{fluence}_fulldam.txt")[:,1],
    ]

    return np.vstack(data).T

factor_block_010 = load_factor_block("010")
factor_block_100 = load_factor_block("100")

column_names_fig4 = [
    "q 1/A",
    "undamaged water",
    "displacement only",
    "formfactor only",
    "full model"
]

(50, 5)

In [31]:
with pd.ExcelWriter("./excel/fig4_Iq_contributions.xlsx") as writer:
    pd.DataFrame(factor_block_010, columns=column_names_fig4).to_excel(writer, sheet_name=f"10% fluence", index=False)
    pd.DataFrame(factor_block_100, columns=column_names_fig4).to_excel(writer, sheet_name=f"100% fluence", index=False)